In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from numpy.random import seed
seed(1)
# from tensorflow import set_random_seed
# set_random_seed(1)
import math
import tensorflow as tf
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from math import sqrt
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
from numpy.ma.core import ceil
from copy import deepcopy
from tensorflow.keras import optimizers
# notebookpath = "/content/drive/MyDrive/Colab Notebooks/ADANIPORTS.csv"
notebookpath = "/content/drive/MyDrive/Colab Notebooks/SMA STD Bol Upper,Lower and also some extra new features.csv"
# notebookpath ="/content/drive/MyDrive/Colab Notebooks/New Adani Datafrae with added features SMA,STD,Bollinger Upper,Bollinger Lower and removed useless features Series and Symbol.csv"
df = pd.read_csv(notebookpath);
# print(help(mean_squared_error))
df
# print(help(tf.keras.optimizers))
# print("-----------------------------------------------------")
# print(help(optimizers))

,Unnamed: 0,Date,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,...,SMA,STD,Bollinger Upper,Bollinger Lower,Momentum,Williams %14,Stochastic %K with window size14,Stochastic %D with window size 3,Symbol,Series
0,0,2008-04-07,593.20,595.0,605.30,570.00,591.80,593.40,590.59,466172,...,562.5100,50.278235,663.066471,461.953529,0.071651,-38.387716,66.288952,43.125700,MUNDRAPORT,EQ
1,1,2008-04-08,593.40,625.0,625.00,585.10,588.00,589.65,590.78,217730,...,561.5050,49.478799,660.462598,462.547402,0.070348,-40.187140,11.403509,35.322774,MUNDRAPORT,EQ
2,2,2008-04-09,589.65,580.5,623.00,575.15,598.90,597.75,605.82,507422,...,561.3350,49.341066,660.017132,462.652868,0.068881,-36.300384,47.230930,41.641130,MUNDRAPORT,EQ
3,3,2008-04-10,597.75,615.0,615.00,586.00,611.00,608.60,609.93,495019,...,560.5800,48.443924,657.467848,463.692152,0.067573,-31.094050,77.931034,45.521824,MUNDRAPORT,EQ
4,4,2008-04-11,608.60,610.0,620.00,605.00,610.75,610.20,612.70,301522,...,560.9450,48.805716,658.556431,463.333569,0.066528,-30.333573,34.666667,53.276210,MUNDRAPORT,EQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,3228,2021-04-26,725.35,733.0,739.65,728.90,729.20,730.75,733.25,9390549,...,745.1675,44.130766,833.429033,656.905967,4.719806,-78.900256,17.209302,41.327657,ADANIPORTS,EQ
3229,3229,2021-04-27,730.75,735.0,757.50,727.35,748.60,749.15,747.67,20573107,...,747.2825,43.201566,833.685633,660.879367,4.637634,-69.488491,72.305141,36.367999,ADANIPORTS,EQ
3230,3230,2021-04-28,749.15,755.0,760.00,741.10,743.40,746.25,751.02,11156977,...,750.1825,40.916578,832.015657,668.349343,4.541741,-70.971867,27.248677,38.921040,ADANIPORTS,EQ
3231,3231,2021-04-29,746.25,753.2,765.85,743.40,746.40,746.75,753.06,13851910,...,752.3675,39.406109,831.179717,673.555283,4.455725,-61.447811,14.922049,38.158622,ADANIPORTS,EQ


In [ ]:
#preprocessing for LSTM
meantrades = df['Trades'].mean()
print(meantrades)
df['Trades'].fillna(value=meantrades, inplace=True)
 
# df = df.drop(['Volume'], axis = 1)
# df = df.drop(['Turnover'], axis = 1)
# df = df.drop(['Trades'], axis = 1)
# df = df.drop(['Deliverable Volume'], axis = 1)
# df = df.drop(['%Deliverble'], axis = 1)
# df = df.drop(['VWAP'], axis = 1)
# df = df.drop(['Last'], axis = 1)
# df = df.drop(['Low'], axis = 1)
df = df.drop(columns = ['Symbol','Series','Volume','VWAP','Deliverable Volume','%Deliverble'], axis = 1)
# df = df.drop(['Prev Close'], axis = 1)

# print(df)
# normalize the dataset
lastcol = len(df.columns)
dfstr = df[df.columns[list(range(0,2))]]
# print(dfstr)
dfnum = df[df.columns[list(range(2,lastcol))]]
# dfnum = df[df.columns[list(range(8,8+1))]]
scaler = MinMaxScaler()
# print(type(dfnum))
cols = list(dfnum.columns)
dfnum = scaler.fit_transform(dfnum)
smin = scaler.data_min_
smax = scaler.data_max_
dfnum = pd.DataFrame(dfnum, columns = cols)
  
# dfscaled = pd.concat([dfstr , dfnum], axis = 1, join = 'inner')
dfscaled = dfnum
print(smin)
print(smax)
print(dfscaled)


44922.58835504897
[ 1.08000000e+02  1.08000000e+02  1.10450000e+02  1.05650000e+02
  1.08000000e+02  1.08000000e+02  2.41585705e+11  3.66000000e+02
  1.12602500e+02  1.50964226e+00  1.17606969e+02 -2.47848474e+02
  6.12916501e-09 -9.96614565e+01  1.38888889e+00  9.94671621e+00]
[ 9.01350000e+02  9.09900000e+02  9.26000000e+02  8.69800000e+02
  9.02000000e+02  9.01350000e+02  8.16098831e+15  1.20598400e+06
  8.15947500e+02  3.31415228e+02  1.21090447e+03  7.62069637e+02
  1.06287078e+01 -1.11448835e+00  1.00000000e+02  8.98280964e+01]
      Prev Close      Open      High       Low      Last     Close  Turnover  \
0       0.611584  0.607308  0.606768  0.607669  0.609320  0.611836  0.003344   
1       0.611836  0.644719  0.630924  0.627429  0.604534  0.607109  0.001547   
2       0.607109  0.589226  0.628472  0.614408  0.618262  0.617319  0.003737   
3       0.617319  0.632248  0.618662  0.628607  0.633501  0.630995  0.003670   
4       0.630995  0.626013  0.624793  0.653471  0.633186  0.

In [ ]:
# window 20, rmse corrected, data 1300:2300
# architecture https://analyticsindiamag.com/hands-on-guide-to-lstm-recurrent-neural-network-for-stock-market-prediction/

le = len(dfscaled)
print(le)

window = [5]
# window = [5,10,15]

def transdata(dfscaled, win):
  df2 = deepcopy(dfscaled)
  col = df2.columns
  for i in range(2, win+1):
    for c in col:
      df2[c+str(i)] = df2[c].shift(-(i-1)) #stepback i-1 times
  
  df2['Close'+str(win+1)] = df2['Close'].shift(-(win)) #stepback i-1 times
  return df2
# print(help(Dense))
highcost = [152.42, 352.35, 619.43]
lowcost = [133.96, 301.55, 464.24]
ind = [(0,900,900,1100,1100,1300),(0,1900,1900, 2100, 2100, 2300),(0, 2800, 2800, 3000, 3000, 3200)]
for index in range(3):
  print('index'+str(index))
  for prevd in window:
    print('prevd'+str(prevd))
    #transforming data
    df2 = transdata(dfscaled, prevd)
    # print(df2)
    df2 = df2.dropna()
    print('len(df2)')
    print(len(df2))

    
    # df2['High'] = 
    # print(df2)

    #split
    dftr = deepcopy(df2.loc[ind[index][0]:ind[index][1]])
    # dftr = deepcopy(df2.loc[1300:2100])
    dftrY = dftr['Close'+str(prevd+1)].to_numpy()  
    dftrX = dftr.drop(['Close'+str(prevd+1)], axis = 1).to_numpy() 

    dfva = deepcopy(df2.loc[ind[index][2]:ind[index][3]])
    # dftr = deepcopy(df2.loc[1300:2100])
    dfvaY = dfva['Close'+str(prevd+1)].to_numpy()  
    dfvaX = dfva.drop(['Close'+str(prevd+1)], axis = 1).to_numpy() 

    dftes = deepcopy(df2.loc[ind[index][4]:ind[index][5]])
    # dftes =  deepcopy(df2.loc[2100:2300])
    dftesY = dftes['Close'+str(prevd+1)].to_numpy()
    dftesX = dftes.drop(['Close'+str(prevd+1)], axis = 1).to_numpy()

    # dftrX,   dftesX = np.reshape(dftrX, (dftrX.shape[0], 1, dftrX.shape[1])),   np.reshape(dftesX, (dftesX.shape[0], 1, dftesX.shape[1]))
    dftrX,  dfvaX, dftesX = np.reshape(dftrX, (dftrX.shape[0],  dftrX.shape[1], 1)), np.reshape(dfvaX, (dfvaX.shape[0],  dfvaX.shape[1], 1)),   np.reshape(dftesX, (dftesX.shape[0],  dftesX.shape[1], 1))
    
    # print(dftrX.shape)
    # print(dftrX.shape[1])
    tsterr, trainerr, valerr = [], [], []
    regressor = Sequential()
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (dftrX.shape[1], 1)))
    # regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50, return_sequences = True))
    # regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50, return_sequences = True))
    # regressor.add(Dropout(0.2))
    regressor.add(LSTM(units = 50))
    # regressor.add(Dropout(0.2))
    regressor.add(Dense(units = 1))
    regressor.compile(loss = 'mean_squared_error', optimizer = 'adam')#adam is better than sgd
    # print(regressor.summary())
     
    leastvalerr = 1e9
    for epoch in range(400):
      #training
      print('epoch')
      print(epoch)
      regressor.fit(dftrX, dftrY, epochs=1, batch_size=32, verbose=2) 
      #predict on testset
      trp = regressor.predict(dftrX)
      valp = regressor.predict(dfvaX)
      
      #inverse transform
      trp =  (trp*(smax[5]-smin[5])) + smin[5]
      valp =  (valp*(smax[5]-smin[5])) + smin[5]
      dfvaYin = (dfvaY*(smax[5]-smin[5])) + smin[5]
      dftrYin = (dftrY*(smax[5]-smin[5])) + smin[5]
      # calculate rmse
      trp = trp.reshape((trp.shape[0],)) 
      trainerr.append(sqrt(mean_squared_error(trp, dftrYin)))
      # trainerr.append(rmsenorm(trp,dftrY))
      valp = valp.reshape((valp.shape[0],)) 
      valerr.append(sqrt(mean_squared_error(valp, dfvaYin)))
      if( leastvalerr > valerr[-1]):
        leastvalerr = valerr[-1]
        regressor.save(str(index)+'my_model.h5')
        # bestmodel = deepcopy(regressor)
      # tsterr.append(rmsenorm(tesp,dftesY))
     
    bestmodel = load_model(str(index)+'my_model.h5')
    print('validation error train error')
    for i in range(len(trainerr)):
      outt = "{:.10f} {:.10f}"
      print(outt.format( float(valerr[i]),  float(trainerr[i])))  
    tep = bestmodel.predict(dftesX)
    #inverse transform
    tep =  (tep*(smax[5]-smin[5])) + smin[5]
    tep = tep.reshape((tep.shape[0],)) 
    dftesYin = (dftesY*(smax[5]-smin[5])) + smin[5]
    print("testerror of best model")
    print(sqrt(mean_squared_error(tep, dftesYin)))
    high,mid,low = [],[],[]
    print(dftesY.shape)
    for i in range(len(dftesYin)):
      # print('dftesY[i]')
      # print(dftesYin[i])
      if(dftesYin[i] > highcost[index] ):
        high.append( (dftesYin[i] - tep[i])**2)
      elif(dftesYin[i] < lowcost[index]):
        low.append( (dftesYin[i] - tep[index])**2)
      else:
        mid.append((dftesYin[i] - tep[index])**2 )
    if(len(high) > 0):
      print('high')
      print(sqrt(sum(high)/len(high)))
    if(len(mid) > 0):
      print('mid')
      print(sqrt(sum(mid)/len(mid)))
    if(len(low) > 0):
      print('low')
      print(sqrt(sum(low)/len(low)))
     
        

3233
index0
prevd5
len(df2)
3228
epoch
0
29/29 - 8s - loss: 0.0536 - 8s/epoch - 286ms/step
epoch
1
29/29 - 2s - loss: 0.0069 - 2s/epoch - 71ms/step
epoch
2
29/29 - 2s - loss: 0.0048 - 2s/epoch - 71ms/step
epoch
3
29/29 - 2s - loss: 0.0045 - 2s/epoch - 71ms/step
epoch
4
29/29 - 2s - loss: 0.0045 - 2s/epoch - 71ms/step
epoch
5
29/29 - 2s - loss: 0.0045 - 2s/epoch - 71ms/step
epoch
6
29/29 - 2s - loss: 0.0046 - 2s/epoch - 71ms/step
epoch
7
29/29 - 2s - loss: 0.0043 - 2s/epoch - 71ms/step
epoch
8
29/29 - 2s - loss: 0.0042 - 2s/epoch - 71ms/step
epoch
9
29/29 - 2s - loss: 0.0048 - 2s/epoch - 70ms/step
epoch
10
29/29 - 2s - loss: 0.0040 - 2s/epoch - 71ms/step
epoch
11
29/29 - 2s - loss: 0.0039 - 2s/epoch - 71ms/step
epoch
12
29/29 - 2s - loss: 0.0040 - 2s/epoch - 71ms/step
epoch
13
29/29 - 2s - loss: 0.0035 - 2s/epoch - 71ms/step
epoch
14
29/29 - 2s - loss: 0.0027 - 2s/epoch - 71ms/step
epoch
15
29/29 - 2s - loss: 0.0025 - 2s/epoch - 71ms/step
epoch
16
29/29 - 2s - loss: 0.0021 - 2s/epoch - 

In [ ]:
# help(LSTM)